### Initialize disk fitting object (D from TMC1A_fitting.py)

In [ ]:
from TMC1A_fitting import *
from analysis import generate_uncertainty_and_effective_beam_size_v2
import emcee
from multiprocess import Pool

### Update observation uncertainty (Xu+2023 Appendix A, Eq. A5)

In [ ]:
# Aso21
inc = 53
pa = 75
cosI = np.cos(inc/180*pi)

for i in range(4):
    I = D.disk_image_list[i]
    I.disk_pa = pa
    generate_uncertainty_and_effective_beam_size_v2(I, cosI)

# MCMC

### fiducial: bound Q to [1,2]

In [ ]:
bounds['Q10'] = [1,2]
bounds['Q100'] = [1,2]
def get_log_probability_constrainQ(theta):
    Rd = np.exp(theta[2+var_names.index('Rd')])/au
    Q10 = np.exp(theta[2+var_names.index('Q10')])
    Q100 = np.exp(theta[2+var_names.index('Q100')])
    Qs = np.log(Q100/Q10)/np.log(10)
    Q5 = Q10*.5**Qs
    Qd = Q10*(Rd/10)**Qs
    if Q5<1 or Q5>2 or Qd<1 or Qd>2:
        return -np.inf
    return get_log_probability(theta, weights=None, Mp_prior=True)
ic = get_MCMC_ic(nwalkers=128,sigma_log_model=True)

In [ ]:
if True:
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        sampler_par = emcee.EnsembleSampler(
            ic.shape[0], ic.shape[1], get_log_probability_constrainQ,
            pool = Pool(8)
        )
        _ = sampler_par.run_mcmc(ic, 100, progress=True)
    pickle.dump(sampler_par, open('./sampler_GI_newimg_w128_s100.pkl','wb'))

In [ ]:
bounds['Q10'] = [1,2]
bounds['Q100'] = [1,2]
if True:
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        sampler_par = pickle.load(open('./sampler_GI_newimg_w128_s100.pkl','rb'))
        sampler_par.pool = Pool(8)
        _ = sampler_par.run_mcmc(None, 400, progress=True)
        pickle.dump(sampler_par, open('./sampler_GI_newimg_w128_s500.pkl','wb'))

In [ ]:
bounds['Q10'] = [1,2]
bounds['Q100'] = [1,2]
if True:
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        sampler_par = pickle.load(open('./sampler_GI_newimg_w128_s500.pkl','rb'))
        sampler_par.pool = Pool(8)
        _ = sampler_par.run_mcmc(None, 500, progress=True)
        pickle.dump(sampler_par, open('./sampler_GI_newimg_w128_s1000.pkl','wb'))

### No assumption on Q

In [ ]:
bounds['Q10'] = [.1,1e3]
bounds['Q100'] = [.1,1e3]
ic = get_MCMC_ic(nwalkers=128,sigma_log_model=True)

In [ ]:
if True:
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        sampler_par = emcee.EnsembleSampler(
            ic.shape[0], ic.shape[1], get_log_probability,
            args=(None,True,),
            pool = Pool(8)
        )
        _ = sampler_par.run_mcmc(ic, 100, progress=True)
        pickle.dump(sampler_par, open('./sampler_noGI_newimg_w128_s100.pkl','wb'))

In [ ]:
if True:
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        sampler_par = pickle.load(open('./sampler_noGI_newimg_w128_s100.pkl','rb'))
        sampler_par.pool = Pool(8)
        _ = sampler_par.run_mcmc(None, 400, progress=True)
        pickle.dump(sampler_par, open('./sampler_noGI_newimg_w128_s500.pkl','wb'))

In [ ]:
if True:
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        sampler_par = pickle.load(open('./sampler_noGI_newimg_w128_s500.pkl','rb'))
        sampler_par.pool = Pool(8)
        _ = sampler_par.run_mcmc(None, 500, progress=True)
        pickle.dump(sampler_par, open('./sampler_noGI_newimg_w128_s1000.pkl','wb'))